In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [4]:
key_path = key_path
project = project_id

# Sources
indicators_table = 'silver_fred_macro_data'
indicators_dataset = 'silver'

# Tables id
table_conca_indicators = f'{project}.{indicators_dataset}.{indicators_table}'

# Loading tables
table_to_save = 'silver_itm_fred_macro_data'
dataset_to_save = 'silver'

In [5]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [6]:
df = bigquery.run_query(
    f"""
    SELECT
        *
    FROM {table_conca_indicators}  
    """
)

df

,date,BOPGSTB,CPIAUCSL,FEDFUNDS,GDP,PPIACO,RSAFS,UMCSENT,UNRATE,id
0,2015-01-01 00:00:00+00:00,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,4c6f3a3ad2c3295df78d540b586fa7a3
1,2015-01-02 00:00:00+00:00,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,d046a975255510ffcbbf9698294fdf40
2,2015-01-03 00:00:00+00:00,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,839cd1c08a392d57d500f00285268f2d
3,2015-01-04 00:00:00+00:00,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,0518f2b63236a9bc3119f02cabae4fd4
4,2015-01-05 00:00:00+00:00,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,71e7b2696f675336f79208247cdc5594
...,...,...,...,...,...,...,...,...,...,...
3525,2024-08-26 00:00:00+00:00,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3,ef84b4618ba298b6579599a9133cb02c
3526,2024-08-27 00:00:00+00:00,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3,d03c334c76ec10668a8689d5490c227f
3527,2024-08-28 00:00:00+00:00,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3,c03f67e2119c230ed5198baf394982cf
3528,2024-08-29 00:00:00+00:00,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3,a2125ebc5a8ad74a15dbf2b516e983c5


In [7]:
df_unpivot = df.melt(id_vars=['id', 'date'], var_name='macro_indicator', value_name='macro_indicator_value')

df_unpivot = df_unpivot.sort_values(by='id')

df_unpivot

,id,date,macro_indicator,macro_indicator_value
27192,000e0b1e9106f81ee4cfbdcb4d2be4c5,2021-10-18 00:00:00+00:00,UNRATE,4.500
23662,000e0b1e9106f81ee4cfbdcb4d2be4c5,2021-10-18 00:00:00+00:00,UMCSENT,71.700
16602,000e0b1e9106f81ee4cfbdcb4d2be4c5,2021-10-18 00:00:00+00:00,PPIACO,240.465
13072,000e0b1e9106f81ee4cfbdcb4d2be4c5,2021-10-18 00:00:00+00:00,GDP,24654.603
6012,000e0b1e9106f81ee4cfbdcb4d2be4c5,2021-10-18 00:00:00+00:00,CPIAUCSL,276.434
...,...,...,...,...
18868,fffb67182eb91933f2141cb47dc43a35,2018-05-03 00:00:00+00:00,RSAFS,500930.000
4748,fffb67182eb91933f2141cb47dc43a35,2018-05-03 00:00:00+00:00,CPIAUCSL,250.792
25928,fffb67182eb91933f2141cb47dc43a35,2018-05-03 00:00:00+00:00,UNRATE,3.800
11808,fffb67182eb91933f2141cb47dc43a35,2018-05-03 00:00:00+00:00,GDP,20580.912


In [8]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df_unpivot, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
